Split Folder

In [ ]:
import os
import shutil
import random
from collections import defaultdict

DATASET_DIR = "./dataset"
OUTPUT_DIR = "./splitted_dataset"
RATIOS = (0.7, 0.15, 0.15)
SEED = 1337

random.seed(SEED)

def get_patient_id(fname):
    return "_".join(fname.split("_")[:2])


patients_per_class = defaultdict(lambda: defaultdict(list))

for cls in os.listdir(DATASET_DIR):
    cls_path = os.path.join(DATASET_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    for fname in os.listdir(cls_path):
        pid = get_patient_id(fname)
        patients_per_class[cls][pid].append(fname)

splits = {"train": [], "val": [], "test": []}

for cls, patient_dict in patients_per_class.items():
    patient_ids = list(patient_dict.keys())
    random.shuffle(patient_ids)

    n = len(patient_ids)

    if n < 2:
        print(f"[WARNING] Class '{cls}' hanya punya {n} patient. Tidak bisa split masuk akal.")
        splits["train"].extend([(cls, pid) for pid in patient_ids])
        continue
    n_train = max(1, int(RATIOS[0] * n))
    n_val   = max(1, int(RATIOS[1] * n)) if n >= 3 else 0

    if n_train + n_val >= n:
        n_train = n - 1
        n_val = 0

    train_ids = patient_ids[:n_train]
    val_ids   = patient_ids[n_train:n_train + n_val]
    test_ids  = patient_ids[n_train + n_val:]

    splits["train"].extend([(cls, pid) for pid in train_ids])
    splits["val"].extend([(cls, pid) for pid in val_ids])
    splits["test"].extend([(cls, pid) for pid in test_ids])

for split, items in splits.items():
    for cls, pid in items:
        out_dir = os.path.join(OUTPUT_DIR, split, cls)
        os.makedirs(out_dir, exist_ok=True)

        for fname in patients_per_class[cls][pid]:
            src = os.path.join(DATASET_DIR, cls, fname)
            dst = os.path.join(out_dir, fname)
            shutil.copy2(src, dst)


# Importing & Configuration

In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
from sklearn.metrics import classification_report, f1_score
import copy
import timm
from tqdm import tqdm
from ultralytics import YOLO

c:\Users\Alvin\Downloads\PythonProjects\RM-Project-Copy(1)\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_DIR = "./splitted_dataset"
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCH = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Data Transformers

In [3]:
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

# Data Loading

In [4]:
image_datasets = {x: datasets.ImageFolder(f"{DATASET_DIR}/{x}", data_transforms[x]) for x in ['train','val']}
dataloaders = {x: DataLoader(image_datasets[x],batch_size=BATCH_SIZE,shuffle=True, num_workers=2) for x in ['train','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes

print(class_names)
print(dataset_sizes)

['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
{'train': 60573, 'val': 12261}


# Preprocessing

In [5]:
y_train = image_datasets['train'].targets
class_counts = np.bincount(y_train)
total_samples = len(y_train)
class_weights = total_samples / (len(class_names) * class_counts)
class_weights = torch.FloatTensor(class_weights).to(DEVICE)

# Training

Training Function

In [6]:
def train_model(model:nn.Module,model_name:str,criterion:nn.CrossEntropyLoss, optimizer:torch.optim.Optimizer, num_epochs = EPOCH):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print("-"*10)

        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            all_preds = []
            all_labels = []

            pbar = tqdm(dataloaders[phase], desc=f'Epoch {epoch+1} {phase}')

            for inputs, labels in pbar:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                pbar.set_postfix({'loss':loss.item()})
                running_loss += loss.item() * inputs.size(0)
                preds = torch.argmax(outputs, dim = 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_f1 = f1_score(all_labels,all_preds,average='macro')

            print(f'{phase} Loss: {epoch_loss:.4f} F1: {epoch_f1:.4f}')

            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(),f'best_models/{model_name}_best.pth')
                print(f'New best model is saved with F1 : {best_f1:.4f}')

    model.load_state_dict(best_model_wts)
    return model


In [14]:
model_to_test_1 = ['resnetv2_50.a1h_in1k', 'convnext_tiny']

In [ ]:
for model_name in model_to_test_1:
    print(f"{'='*10}")
    print(f"{model_name}'s Training")
    print(f'{'='*10}')

    model = timm.create_model(model_name,pretrained=True,num_classes = NUM_CLASSES)
    model = model.to(DEVICE)

    criteration = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)
    train_model(model,model_name,criteration,optimizer,num_epochs=EPOCH)

resnetv2_50.a1h_in1k's Training
Epoch 1/20
----------


Epoch 1 train: 100%|██████████| 1893/1893 [04:54<00:00,  6.43it/s, loss=0.503]


train Loss: 0.9214 F1: 0.4735


Epoch 1 val: 100%|██████████| 384/384 [01:34<00:00,  4.04it/s, loss=5.43]


val Loss: 5.9803 F1: 0.0994
New best model is saved with F1 : 0.0994
Epoch 2/20
----------


...

train Loss: 0.0153 F1: 0.9781


Epoch 20 val: 100%|██████████| 384/384 [01:17<00:00,  4.92it/s, loss=19]  


val Loss: 17.0621 F1: 0.0585
xception's Training
Epoch 1/20
----------


Epoch 1 train:   2%|▏         | 43/1893 [02:09<1:32:53,  3.01s/it, loss=1.14] 


KeyboardInterrupt: 

In [7]:
model_to_test_2 = ['efficientnet_b0']

In [127]:
for model_name in model_to_test_2:
    print(f"{'='*10}")
    print(f"{model_name}'s Training")
    print(f'{'='*10}')

    model = timm.create_model(model_name,pretrained=True,num_classes = NUM_CLASSES)
    model = model.to(DEVICE)

    criteration = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)
    train_model(model,model_name,criteration,optimizer,num_epochs=EPOCH)

efficientnet_b0's Training
Epoch 1/20
----------


Epoch 1 train: 100%|██████████| 1893/1893 [07:21<00:00,  4.29it/s, loss=0.116]  


train Loss: 0.4800 F1: 0.6361


Epoch 1 val: 100%|██████████| 384/384 [01:31<00:00,  4.19it/s, loss=20.7]


val Loss: 16.1472 F1: 0.0618
New best model is saved with F1 : 0.0618
Epoch 2/20
----------


...

val Loss: 35.3338 F1: 0.0895
Epoch 20/20
----------


Epoch 20 train: 100%|██████████| 1893/1893 [06:54<00:00,  4.57it/s, loss=1.19e-5] 


train Loss: 0.0040 F1: 0.9983


Epoch 20 val: 100%|██████████| 384/384 [00:56<00:00,  6.82it/s, loss=50.5]

val Loss: 36.5110 F1: 0.0556


# Evaluate

In [8]:
def evaluate_on_test_set(model,model_name):
    state_dict = torch.load(f'best_models/{model_name}_best.pth')
    model.load_state_dict(state_dict)

    model.eval()

    all_preds = []
    all_labels = []

    test_dataset = datasets.ImageFolder(f'{DATASET_DIR}/test',data_transforms['val'])
    test_loader = DataLoader(test_dataset,batch_size =BATCH_SIZE,shuffle=False)

    with torch.no_grad():
        for inputs,labels in test_loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            preds = torch.argmax(outputs,dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
    print(classification_report(all_labels,all_preds, target_names=test_dataset.classes))

In [15]:
model_to_evaluate = model_to_test_1
print(model_to_evaluate)

['resnetv2_50.a1h_in1k', 'convnext_tiny']


In [ ]:
for model_name in model_to_evaluate:
    print(f"Evaluation Report {model_name}")
    model = timm.create_model(model_name,pretrained=True,num_classes = NUM_CLASSES)
    model = model.to(DEVICE)

    evaluate_on_test_set(model,model_name)

Evaluation Report resnetv2_50.a1h_in1k
                    precision    recall  f1-score   support

     Mild Dementia       0.24      0.15      0.19       976
 Moderate Dementia       0.00      0.00      0.00       244
      Non Demented       0.81      0.92      0.86     10004
Very mild Dementia       0.37      0.26      0.30      2379

          accuracy                           0.73     13603
         macro avg       0.35      0.33      0.34     13603
      weighted avg       0.68      0.73      0.70     13603

Evaluation Report convnext_tiny
                    precision    recall  f1-score   support

     Mild Dementia       0.09      0.13      0.11       976
 Moderate Dementia       0.02      0.20      0.04       244
      Non Demented       0.75      0.73      0.74     10004
Very mild Dementia       0.00      0.00      0.00      2379

          accuracy                           0.55     13603
         macro avg       0.22      0.27      0.22     13603
      weighted avg      

In [ ]:
model_to_evaluate = model_to_test_2

In [131]:
for model_name in model_to_evaluate:
    print(f"Evaluation Report {model_name}")
    model = timm.create_model(model_name,pretrained=True,num_classes = NUM_CLASSES)
    model = model.to(DEVICE)

    evaluate_on_test_set(model,model_name)

Evaluation Report efficientnet_b0
                    precision    recall  f1-score   support

     Mild Dementia       0.31      0.12      0.17       976
 Moderate Dementia       0.00      0.00      0.00       244
      Non Demented       0.78      0.92      0.85     10004
Very mild Dementia       0.32      0.19      0.24      2379

          accuracy                           0.72     13603
         macro avg       0.35      0.31      0.31     13603
      weighted avg       0.65      0.72      0.68     13603



In [10]:
BEST_MODEL = 'resnetv2_50.a1h_in1k'

# Resampling

In [ ]:
import os
import shutil
import random
from collections import defaultdict

DATASET_DIR = "./dataset"
OUTPUT_DIR = "./splitted_dataset_resample"
RATIOS = (0.7, 0.15, 0.15)
SEED = 1337

random.seed(SEED)

def get_patient_id(fname):
    return "_".join(fname.split("_")[:2])


patients_per_class = defaultdict(lambda: defaultdict(list))

for cls in os.listdir(DATASET_DIR):
    cls_path = os.path.join(DATASET_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    for fname in os.listdir(cls_path):
        pid = get_patient_id(fname)
        patients_per_class[cls][pid].append(fname)

splits = {"train": [], "val": [], "test": []}

for cls, patient_dict in patients_per_class.items():
    patient_ids = list(patient_dict.keys())
    random.shuffle(patient_ids)

    n = len(patient_ids)

    if n < 2:
        print(f"[WARNING] Class '{cls}' hanya punya {n} patient. Tidak bisa split masuk akal.")
        splits["train"].extend([(cls, pid) for pid in patient_ids])
        continue
    if n == 2:
        n_train, n_val = 1, 0
    elif n == 3:
        n_train,n_val = 1,1
    else:
        n_train = int(RATIOS[0] * n)
        n_val   = int(RATIOS[1] * n)
    
    if n_train<1:n_train = 1
    if n_val< 1:n_val = 1

    if n_train + n_val >= n:
        n_train = n - n_val

    train_ids = patient_ids[:n_train]
    val_ids   = patient_ids[n_train:n_train + n_val]
    test_ids  = patient_ids[n_train + n_val:]

    splits["train"].extend([(cls, pid) for pid in train_ids])
    splits["val"].extend([(cls, pid) for pid in val_ids])
    splits["test"].extend([(cls, pid) for pid in test_ids])

for split, items in splits.items():
    for cls, pid in items:
        out_dir = os.path.join(OUTPUT_DIR, split, cls)
        os.makedirs(out_dir, exist_ok=True)

        for fname in patients_per_class[cls][pid]:
            src = os.path.join(DATASET_DIR, cls, fname)
            dst = os.path.join(out_dir, fname)
            shutil.copy2(src, dst)


In [ ]:
DATASET_DIR = "./splitted_dataset_resample"
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCH = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

In [11]:
#ensuring the data value
print(class_weights)
print(len(class_weights))

print(BATCH_SIZE)

tensor([ 4.5972, 62.0625,  0.3183,  1.6016], device='cuda:0')
4
32


In [12]:
sample_weights = [class_weights[label] for label in y_train]
print(sample_weights)

[tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.5972, device='cuda:0'), tensor(4.

In [16]:
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

In [13]:
resampled_loader = DataLoader(
    image_datasets['train'],
    batch_size=BATCH_SIZE,
    sampler=sampler,
    num_workers=2
)

In [15]:
model = timm.create_model(BEST_MODEL)
model = model.to(DEVICE)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
def train_resample_model(model:nn.Module,model_name:str,criterion:nn.CrossEntropyLoss, optimizer:torch.optim.Optimizer, num_epochs = EPOCH):
    best_f1 = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print("-"*10)

        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            all_preds = []
            all_labels = []

            pbar = tqdm(resampled_loader, desc=f'Epoch {epoch+1} {phase}')

            for inputs, labels in pbar:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                pbar.set_postfix({'loss':loss.item()})c
                running_loss += loss.item() * inputs.size(0)
                preds = torch.argmax(outputs, dim = 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_f1 = f1_score(all_labels,all_preds,average='macro')

            print(f'{phase} Loss: {epoch_loss:.4f} F1: {epoch_f1:.4f}')

            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(),f'best_models/{model_name}_resample_best.pth')
                print(f'New best model is saved with F1 : {best_f1:.4f}')

    model.load_state_dict(best_model_wts)
    return model


In [18]:
train_resample_model(model,BEST_MODEL,criterion,optimizer,EPOCH)

Epoch 1/20
----------


Epoch 1 train: 100%|██████████| 1/1 [00:12<00:00, 12.57s/it, loss=7.11]


train Loss: 0.0005 F1: 0.0000


Epoch 1 val: 100%|██████████| 1/1 [00:09<00:00,  9.66s/it, loss=0.547]


val Loss: 0.0002 F1: 1.0000
New best model is saved with F1 : 1.0000
Epoch 2/20
----------


Epoch 2 train:   0%|          | 0/1 [00:03<?, ?it/s]


KeyboardInterrupt: 